

```
# This is formatted as code
```

# DMA Fall 24

**Note** : This entire lab will be manually evaluated.

In [1]:
NAME = "Bingbing Ma"
COLLABORATORS = ""

---

# Lab 4: Neural Networks #


In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
!wget -nc http://quadro.ist.berkeley.edu:1331/lab_4_training.csv
!wget -nc http://quadro.ist.berkeley.edu:1331/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2024-10-01 19:49:44--  http://quadro.ist.berkeley.edu:1331/lab_4_training.csv
Resolving quadro.ist.berkeley.edu (quadro.ist.berkeley.edu)... 169.229.194.98
Connecting to quadro.ist.berkeley.edu (quadro.ist.berkeley.edu)|169.229.194.98|:1331... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79177 (77K) [text/csv]
Saving to: ‘lab_4_training.csv’

lab_4_training.csv  100%[===================>]  77.32K   171KB/s    in 0.5s    

2024-10-01 19:49:45 (171 KB/s) - ‘lab_4_training.csv’ saved [79177/79177]

--2024-10-01 19:49:45--  http://quadro.ist.berkeley.edu:1331/lab_4_test.csv
Resolving quadro.ist.berkeley.edu (quadro.ist.berkeley.edu)... 169.229.194.98
Connecting to quadro.ist.berkeley.edu (quadro.ist.berkeley.edu)|169.229.194.98|:1331... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26519 (26K) [text/csv]
Saving to: ‘lab_4_test.csv’

lab_4_test.csv      100%[===================>]  25.90K   172KB/s    in 0.2s    

2024-10-01 19:49:46 (172 KB/s) - ‘l

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0
1,677,male,19,second,hazel,72.0,120.0,1,1,16.0,Yes,9.0,70.0,3.0,5.0
2,1738,male,20,second,brown,63.0,55.0,1,2,15.0,Yes,4.5,15.0,4.0,13.0
3,1355,male,20,third,green,78.0,200.0,0,0,10.0,Yes,9.0,20.0,10.0,10.0
4,891,female,19,second,green,67.0,280.0,2,0,4.0,Yes,2.0,164.0,0.0,2.0


In [4]:
df_test.head()

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,1303,male,20,second,green,73.0,210.0,0,1,10.0,Yes,5.0,50.0,1.0,15.0
1,36,male,20,third,other,71.0,90.0,1,0,15.0,Yes,4.0,10.0,0.0,1.0
2,489,male,22,fourth,hazel,75.0,200.0,0,1,1.0,Yes,2.0,150.0,1.0,10.0
3,1415,male,19,second,brown,72.0,35.0,2,2,20.0,Yes,5.0,100.0,0.0,7.0
4,616,male,22,fourth,hazel,71.0,15.0,2,1,10.0,Yes,7.0,10.0,0.0,5.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of `gender`. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives).

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [5]:
# YOUR CODE HERE
gender_train_count = df_train['gender'].value_counts()
max(gender_train_count) / gender_train_count.sum()

0.5427852348993288

###ANSWER: The majoirty  class in the training set is Female. The accuracy would be 0.5427852348993288.

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [6]:
# YOUR CODE HERE
gender_test_count = df_test['gender'].value_counts()
gender_test_count[0] / gender_test_count.sum()

0.5226130653266332

###ANSWER: The accuracy is 0.5226130653266332.

***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (`sigmoid`/`logistic`) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (`lbfgs`). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the `height` feature of the dataset to predict the `gender`. You will have to change `gender` to a 0 and 1 class. After training, use your trained model to predict the class (`gender`) using the `height` feature from the training set. What is the accuracy of this prediction?

In [16]:
# YOUR CODE HERE
X = df_train[['height']]
# df_train = pd.get_dummies(df_train, columns=['gender'])
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)
label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)
clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    # max_iter=100,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

0.8422818791946308

###ANSWER: The accuracy of training set is 0.84.

**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the `height` feature values from the test set. What is the accuracy of this model on the test set?

In [17]:
# YOUR CODE HERE

# clf.fit(X_train, Y_train)

y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test_target, y_pred_test))

0.8489932885906041


###ANSWER: The accuracy of on the test set is 0.85.

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the `height` feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature.

In [19]:
# YOUR CODE HERE

X = np.log(df_train[['height']])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)
# X_train = np.log(X_train)
# X_test = np.log(X_test)
label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    # max_iter=100,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

training set accuracy:  0.8478747203579419
testing set accuracy:  0.8322147651006712


###ANSWER: training set accuracy:  0.8478747203579419; testing set accuracy:  0.8322147651006712


***

### Question 3###
The rest of features in this dataset except a few are categorical. No ML method accepts categorical features, so transform `year`, `eyecolor`, `exercise` into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [20]:
# YOUR CODE HERE

# X = df_train[['year', 'eyecolor', 'exercise']]
X = pd.get_dummies(df_train[['year', 'eyecolor', 'exercise']])
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)
label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.5134228187919463


###ANSWER: testing set accuracy:  0.51

***
### Question 4###
Using a NN, report the accuracy on the test set of a model that trained only on `height` and the `eyecolor` features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original `height` values (no pre-processing) and `eyecolor` as a one-hot?

In [22]:
# YOUR CODE HERE
X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['height']], X_], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)
label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.825503355704698


###ANSWER: testing set accuracy:  0.82

**Question 4.b**  
What is the accuracy on the test set using the log of `height` values (applied to both training and testing sets) and `eyecolor` as a one-hot?

In [23]:
# YOUR CODE HERE
X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['height']], X_], axis = 1)
X['height'] = np.log(X['height'])
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)


label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.8489932885906041


###ANSWER: testing set accuracy:  0.85

**Question 4.c**  
What is the accuracy on the test set using the Z-score of `height` values and `eyecolor` as a one-hot?

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [24]:
X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['height']], X_], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)

standard = StandardScaler()
X_train[['height']] = standard.fit_transform(X_train[['height']])
X_test[['height']] = standard.transform(X_test[['height']])

label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)
clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)


testing set accuracy:  0.8590604026845637


###ANSWER: testing set accuracy:  0.859

***
### Question 5 ###
Repeat question 4 for `watchtv` & `eyecolor`.

**Question 5.a**  
What is the accuracy on the test set using the original `watchtv` values (no pre-processing) and `eyecolor` as a one-hot?

In [27]:
# YOUR CODE HERE

X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['watchtv']], X_], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)


label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.5570469798657718


###ANSWER:testing set accuracy:  0.56

**Question 5.b**  \\
What is the accuracy on the test set using the log of `watchtv` values (applied to both training and testing sets) and `eyecolor` as a one-hot?

Comment: You can drop rows with 0 in the `watchtv` column before applying the log function in order to avoid -inf values.

In [38]:
# YOUR CODE HERE
df_train = df_train[df_train['watchtv'] > 0]
X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['watchtv']], X_], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)

X_train['watchtv'] = np.log(X_train['watchtv'])
X_test['watchtv'] = np.log(X_test['watchtv'])

label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.5567010309278351


###ANSWER: testing set accuracy:  0.56


**Question 5.c** \\
What is the accuracy on the test set using the Z-score of `watchtv` values and `eyecolor` as a one-hot?

In [40]:
# YOUR CODE HERE
X_ = pd.get_dummies(df_train[['eyecolor']])
X = pd.concat([df_train[['watchtv']], X_], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)

# X_train['watchtv'] = (X_train['watchtv'] - X_train['watchtv'].mean()) / (X_train['watchtv']).std()
# X_test['watchtv'] = (X_test['watchtv'] - X_train['watchtv'].mean()) / (X_train['watchtv']).std()
standard = StandardScaler()

X_train['watchtv'] = standard.fit_transform(X_train[['watchtv']])
X_test['watchtv'] = standard.transform(X_test[['watchtv']])

label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.5292096219931272


###ANSWER: testing set accuracy:  0.53


***
### Question 6###
Combine the features from question 3, 4, and 5 (`year`, `eyecolor`, `exercise`, `height`, `watchtv`). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [48]:
# YOUR CODE HERE
X_dummy = pd.get_dummies(df_train[['year', 'eyecolor', 'exercise']])
X = pd.concat([df_train[['watchtv', 'height']], X_dummy], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)
# X_train['watchtv'] = (X_train['watchtv'] - X_train['watchtv'].mean()) / (X_train['watchtv']).std()
# X_test['watchtv'] = (X_test['watchtv'] - X_train['watchtv'].mean()) / (X_train['watchtv']).std()
# X_train['height'] = (X_train['height'] - X_train['height'].mean()) / X_train['height'].std()
# X_test['height'] = (X_test['height'] - X_train['height'].mean()) / X_train['height'].std()

standard = StandardScaler()
X_train[['watchtv', 'height']] = standard.fit_transform(X_train[['watchtv', 'height']])
X_test[['watchtv', 'height']] = standard.transform(X_test[['watchtv', 'height']])

label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='logistic',
                    max_iter=50,           #Default max_iter is 200
                    solver='lbfgs',
                    verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

training = accuracy_score(Y_train_target, y_pred_train) #Accuracy on training set

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
# print('training set accuracy: ', training)
print('testing set accuracy: ', testing)

testing set accuracy:  0.7903780068728522


###ANSWER: 0.79

***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf.

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [59]:
# YOUR CODE HERE
X_dummy = pd.get_dummies(df_train[['year', 'eyecolor', 'exercise']])
X = pd.concat([df_train[['watchtv', 'height']], X_dummy], axis = 1)
Y = df_train['gender']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)

standard = StandardScaler()
X_train[['watchtv', 'height']] = standard.fit_transform(X_train[['watchtv', 'height']])
X_test[['watchtv', 'height']] = standard.transform(X_test[['watchtv', 'height']])

label = LabelEncoder()
label_train = label.fit(Y_train)
Y_train_target = label_train.transform(Y_train)
Y_test_target = label.transform(Y_test)

clf = MLPClassifier(hidden_layer_sizes=(150,100,25),
                    activation='logistic',
                    max_iter=300,           #Default max_iter is 200
                    learning_rate_init=0.001,
                    # verbose=1,              #Default verbose is False
                    random_state=42)        #Default n_iter_no_change=10
clf.fit(X_train, Y_train_target)
y_pred_train=clf.predict(X_train)

y_pred_test=clf.predict(X_test)
testing = accuracy_score(Y_test_target, y_pred_test) #Accuracy on testing set
print('testing set accuracy: ', testing)

testing set accuracy:  0.8694158075601375


###ANSWER: I modified hidden_layer_sizes where I added hidden layers and change the number of nodes in each layer, and added learning_rate in which I chose lower learning-rate since it would converge to a better solution.